# Importing Libraries

In [142]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Get Data from the fulldataset source

This data set has undergone the preliminary cleaning from the python packages, dataprep.py

In [143]:
# Importing Files
fulldataset_df = pd.read_csv('../road_to_qatar_2022/data/fulldataset.csv')
data_cups = pd.read_csv('../raw_data/data/FIFA World Cup/WorldCups.csv')
teams_ranking_df = pd.read_csv('../road_to_qatar_2022/data/teamsranking.csv')

# Slearn Libraries

In [144]:
# Analysis of Worldcup Matches
# Slearn Libraries
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

# Count champions 

In [145]:
# Count champions 
winners =  data_cups['Winner'].map(lambda n: 'Germany' if n == 'Germany FR' else n).value_counts()


# Home and Away Team champions
#data_matches_to_use["Home Team Champion"] = 0
#data_matches_to_use["Away Team Champion"] = 0
fulldataset_df["Home Team Champion"] = 0
fulldataset_df["Away Team Champion"] = 0

def winners_wc(df):
    if(winners.get(df['Home Team Name']) != None):
        df['Home Team Champion'] = winners.get(df['Home Team Name'])
    if(winners.get(df['Away Team Name']) != None):
        df['Away Team Champion'] = winners.get(df['Away Team Name'])
    
    return df

#data_matches_to_use = data_matches_to_use.apply(winners_wc, axis = 1)
fulldataset_df = fulldataset_df.apply(winners_wc, axis = 1)


In [146]:
winners.head()

Brazil       5
Italy        4
Germany      4
Uruguay      2
Argentina    2
Name: Winner, dtype: int64

In [147]:
fulldataset_df.head()

,Date,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion
0,1930-07-13,France,4.0,1.0,Mexico,1,4,1759.78,1764.85,13.0,1644.89,1649.57,1,0
1,1930-07-13,USA,3.0,0.0,Belgium,1,16,1627.48,1635.01,2.0,1816.71,1821.92,0,0
2,1930-07-14,Yugoslavia,2.0,1.0,Brazil,1,6,64.00,64.00,1.0,1841.30,1837.56,0,5
3,1930-07-14,Romania,3.0,1.0,Peru,1,53,1434.68,1427.84,23.0,1561.01,1562.24,0,0
4,1930-07-15,Argentina,1.0,0.0,France,1,3,1773.88,1770.65,4.0,1759.78,1764.85,2,1


In [148]:
fulldataset_df[fulldataset_df['Away Team Name']=='Qatar']

,Date,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion
894,1984-03-28,Iraq,1.0,1.0,Qatar,0,68,1343.01,1338.91,50.0,1439.89,1441.97,0,0
902,1985-07-10,Bahrain,1.0,1.0,Qatar,0,85,1280.64,1289.25,50.0,1439.89,1441.97,0,0
903,1985-07-12,Saudi Arabia,0.0,0.0,Qatar,0,51,1437.78,1435.74,50.0,1439.89,1441.97,0,0


# Replace Team names by corresponding id:

## Re creating the list of teams since the dataset has changed as a result of the mrege with international matches

In [149]:
# Creating a list of all teams playing football worlcup:
team_name = {}
index = 0
for idx, row in fulldataset_df.iterrows():
    name = row['Home Team Name']
    if (name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
    name = row['Away Team Name']
    if (name not in team_name.keys()):
        team_name[name] = index
        index += 1

In [150]:
# Replace Team names by corresponding id:
def replace_name(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    
    return df

#Team_id = data_matches_to_use.apply(replace_name, axis = 1)
Team_id = fulldataset_df.apply(replace_name, axis = 1)

##  Now we dont need number of goals and year

In [151]:
Team_id = Team_id.drop(['Date','Home Team Goals', 'Away Team Goals'], 1)

/tmp/ipykernel_30404/953902738.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Team_id = Team_id.drop(['Date','Home Team Goals', 'Away Team Goals'], 1)


In [152]:
Team_id= Team_id[['Home Team Name',
                 'Away Team Name',
                 'home_team_rank',
                 'home_team_points',
                 'home_team_previous_points',
                 'away_team_rank',
                 'away_team_points',
                 'away_team_previous_points',
                 'Home Team Champion',
                 'Away Team Champion',
                 'Winner']]

In [153]:
Team_id=Team_id.fillna(0)
Team_id.isnull().sum().sum()

0

# Preparing Training and Testing Data:

In [154]:
# Preparing Training and Testing Data:
X = Team_id.iloc[:, 0:10].values
y = Team_id.iloc[:, -1].values

X = np.array(X, dtype = 'f')
y = np.array(y, dtype = 'f')

from sklearn.preprocessing import LabelEncoder
LabelEncoder_res=LabelEncoder()
y=LabelEncoder_res.fit_transform(y)

X, y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Scalling of Data:

In [155]:
# Scalling of Data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Building ANN

In [156]:
# Building ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, confusion_matrix

# seed

In [157]:
#seed
np.random.seed(47)

# Random Forest

In [158]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(n_estimators=100)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.9188861985472155
0.53954802259887
              precision    recall  f1-score   support

           0       0.54      0.59      0.56       147
           1       0.58      0.61      0.59       154
           2       0.34      0.19      0.24        53

    accuracy                           0.54       354
   macro avg       0.49      0.46      0.47       354
weighted avg       0.53      0.54      0.53       354

[[87 50 10]
 [51 94  9]
 [24 19 10]]


# XGBoost

In [159]:
import xgboost as xgb

In [160]:
X_test

array([[-0.11936592, -0.6809748 , -0.7290211 , ...,  0.43665165,
        -0.11888199, -0.39425537],
       [-1.1777142 , -0.8629106 , -0.7598874 , ..., -0.07292214,
        -0.11888199, -0.39425537],
       [-0.4329506 ,  0.41064063, -0.543823  , ...,  0.26016986,
         1.5872352 , -0.39425537],
       ...,
       [-0.51134676, -0.93568504, -0.63642204, ...,  0.13711885,
        -0.6875877 , -0.39425537],
       [-0.7073372 , -0.46265167, -0.451224  , ..., -4.0531898 ,
         0.4498237 , -0.39425537],
       [ 0.3510111 ,  1.4294816 , -0.20429331, ..., -0.20406789,
        -0.6875877 , -0.39425537]], dtype=float32)

In [161]:
# XGBoost

XGB = xgb.XGBClassifier(base_score=0.5, 
                        booster='gbtree', 
                        colsample_bylevel=1,
                        colsample_bytree=0.8, 
                        gamma=0.4, 
                        learning_rate=0.01,
                        max_delta_step=0, 
                        max_depth=3, 
                        min_child_weight=1, 
                        #missing=null,
                        n_estimators=40, 
                        n_jobs=1, 
                        nthread=None, 
                        objective='multi:softprob',
                        random_state=0, 
                        reg_alpha=1e-05, 
                        reg_lambda=1, 
                        scale_pos_weight=1,
                        seed=2, 
                        silent=True, 
                        subsample=0.8)
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[09:39:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.6246973365617433
0.596045197740113
              precision    recall  f1-score   support

           0       0.71      0.45      0.55       147
           1       0.56      0.89      0.69       154
           2       0.44      0.15      0.23        53

    accuracy                           0.60       354
   macro avg       0.57      0.50      0.49       354
weighted avg       0.61      0.60      0.56       354

[[ 66  72   9]
 [ 16 137   1]
 [ 11  34   8]]


# Create the parameters list you wish to tune

In [162]:
parameters = { 'learning_rate' : [0.001,0.01, 0.1, 1],
               'n_estimators' : [40, 100],
               'max_depth': [3, 6],
               'min_child_weight': [1, 3],
               'gamma':[0.4],
               'subsample' : [0.5, 0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             } 

clf = XGB

grid_obj = GridSearchCV(clf,
                        param_grid=parameters,
                        cv=5)
grid_obj = grid_obj.fit(X_train,y_train)
clf = grid_obj.best_estimator_
print(clf)


[09:39:57] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:57] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:57] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:57] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:57] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:58] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:58] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:58] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:58] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:39:58] WARNING: ../src/learner.cc:767: 
Parameters: { "scale

[09:40:12] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:12] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:12] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:13] WARNING: ../src/learner.cc:767: 
Parameters: { "scale

[09:40:26] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:27] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:27] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:27] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:27] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:27] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:28] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:28] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:28] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:28] WARNING: ../src/learner.cc:767: 
Parameters: { "scale

[09:40:40] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:40] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale

[09:40:53] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:53] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:54] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:54] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

[09:40:54] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.001, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max

In [163]:
tuned_xgb_params= clf.get_params()
tuned_xgb_params

{'objective': 'multi:softprob',
 'use_label_encoder': None,
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.8,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0.4,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.001,
 'max_bin': 256,
 'max_cat_threshold': 64,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 40,
 'n_jobs': 1,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 1e-05,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 0.5,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'nthread': 1,
 'seed': 2,
 'silent': True}

In [164]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf_1 = rc.RandomForestClassifier()
clf_1 = rc

grid_obj1 = GridSearchCV(estimator = clf_1, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_obj1 = grid_obj1.fit(X_train,y_train)
clf_1 = grid_obj1.best_estimator_
print(clf_1)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   2.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   1.1s
[C

[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   6.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=300; total time=   1.7s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=   6.7s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=300; total time=   2.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=1000; total time=   5.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_sam

[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_sam

[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time=   6.5s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=300; total time=   1.7s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=1000; total time=   5.3s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, max_features=3, min_sa

[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=8, n_estimators=1000; total time=   5.4s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time=   5.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_sampl

[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=12, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   5.0s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   5.4s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=   5.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=1000; total time=   5.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_s

[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_sa

RandomForestClassifier(max_depth=110, max_features=3, min_samples_leaf=5,
                       min_samples_split=10)


In [184]:
RFC_tuned_params=clf_1.get_params()
RFC_tuned_params

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 110,
 'max_features': 3,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Model with new parameters

In [185]:
# XGBoost
      
XGB = xgb.XGBClassifier(**tuned_xgb_params)                      
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[09:50:07] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.7322751322751323
0.5932203389830508
              precision    recall  f1-score   support

           0       0.57      0.59      0.58        85
           1       0.63      0.73      0.67       113
           2       0.44      0.21      0.29        38

    accuracy                           0.59       236
   macro avg       0.55      0.51      0.51       236
weighted avg       0.58      0.59      0.58       236

[[50 32  3]
 [24 82  7]
 [14 16  8]]


In [186]:
XGB.get_params()

{'objective': 'multi:softprob',
 'use_label_encoder': None,
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.8,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0.4,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.001,
 'max_bin': 256,
 'max_cat_threshold': 64,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 40,
 'n_jobs': 1,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 1e-05,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 0.5,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'nthread': 1,
 'seed': 2,
 'silent': True}

In [187]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
   
rc = RandomForestClassifier(**RFC_tuned_params)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.801058201058201
0.6313559322033898
              precision    recall  f1-score   support

           0       0.60      0.65      0.62        85
           1       0.66      0.73      0.69       113
           2       0.57      0.32      0.41        38

    accuracy                           0.63       236
   macro avg       0.61      0.56      0.57       236
weighted avg       0.63      0.63      0.62       236

[[55 26  4]
 [26 82  5]
 [10 16 12]]


In [188]:
rc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 110,
 'max_features': 3,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# Group Stage - all Kick-off

In [189]:
teams_ranking_df.head()

,rank,countryCode,countryName,totalPoints,previousPoints
0,1,BRA,Brazil,1841.30,1837.56
1,2,BEL,Belgium,1816.71,1821.92
2,3,ARG,Argentina,1773.88,1770.65
3,4,FRA,France,1759.78,1764.85
4,5,ENG,England,1728.47,1737.46


In [190]:
ReWrite_pred_df=pd.DataFrame(columns = ['Home Team Name',
                                        'Away Team Name',
                                        'home_team_rank',
                                        'home_team_points',
                                        'home_team_previous_points',
                                        'away_team_rank',
                                        'away_team_points',
                                        'away_team_previous_points',
                                        'Home Team Champion',
                                        'Away Team Champion',
                                        'Winner'])
ReWrite_pred_df.head()

,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner


In [191]:

def prediction(team1, team2,df):
  ReWrite_pred_df=df
  id1 = team_name[team1]
  id2 = team_name[team2]
  championship1 = winners.get(team1) if winners.get(team1) != None else 0
  championship2 = winners.get(team2) if winners.get(team2) != None else 0
  
  team1_rank_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['rank']
  team1_totalPoints_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['totalPoints']
  team1_previousPoints_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['previousPoints']   

  team2_rank=teams_ranking_df[teams_ranking_df['countryName']==team2]['rank']
  team2_totalPoints=teams_ranking_df[teams_ranking_df['countryName']==team2]['totalPoints']
  team2_previousPoints=teams_ranking_df[teams_ranking_df['countryName']==team2]['previousPoints']   
  
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
  t = np.reshape(t, (1,-1))
  y_rc = XGB.predict_proba(t)[0]
    
  home_team_rank=teams_ranking_df[teams_ranking_df['countryName']==team1]['rank']
  home_team_points=teams_ranking_df[teams_ranking_df['countryName']==team1]['totalPoints']
  home_team_previous_points=teams_ranking_df[teams_ranking_df['countryName']==team1]['previousPoints']
  away_team_rank=teams_ranking_df[teams_ranking_df['countryName']==team2]['rank']
  away_team_points=teams_ranking_df[teams_ranking_df['countryName']==team2]['totalPoints']
  away_team_previous_points=teams_ranking_df[teams_ranking_df['countryName']==team2]['previousPoints']
  Home_Team_Champion=(winners.get(team1) if winners.get(team1) != None else 0)
  Away_Team_Champion=(winners.get(team2) if winners.get(team2) != None else 0)
  conditions = [y_rc[0]> y_rc[1] and y_rc[0]> y_rc[2],
               y_rc[1]> y_rc[2],
               y_rc[2]>y_rc[1]]
  values =[0,1,2]

  Winner=np.select(conditions,values)

  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,
                               'Away Team Name':id2,
                               'home_team_rank':home_team_rank.values[0],
                               'home_team_points':home_team_points.values[0],
                               'home_team_previous_points':home_team_previous_points.values[0],
                               'away_team_rank':away_team_rank.values[0],
                               'away_team_points':away_team_points.values[0],
                               'away_team_previous_points':away_team_previous_points.values[0],
                               'Home Team Champion':Home_Team_Champion,
                               'Away Team Champion':Away_Team_Champion,
                               'Winner':Winner},
                               ignore_index = True)

  text = ('Chance for '+team1+' to win against '+team2+' is {}\nChance for '+team2+' to win against '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(y_rc[1]*100,y_rc[2]*100,y_rc[0]*100)
  return y_rc, text,ReWrite_pred_df

In [192]:
probs, text, ReWrite_pred_df = prediction('Senegal', 'Portugal',ReWrite_pred_df)
print(text)
ReWrite_pred_df.head()

Chance for Senegal to win against Portugal is 33.8610976934433
Chance for Portugal to win against Senegal is 32.955577969551086
Chance for Senegal and Portugal draw is 33.18332135677338


/tmp/ipykernel_30404/3334352355.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
/tmp/ipykernel_30404/3334352355.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
0,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,1


In [193]:
Team_id_updated = Team_id.append(ReWrite_pred_df,ignore_index = True)
Team_id_updated.tail()

/tmp/ipykernel_30404/2971977594.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Team_id_updated = Team_id.append(ReWrite_pred_df,ignore_index = True)


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
1176,65,15,18,1584.38,1584.59,39.0,1486.44,1482.63,0,0,0
1177,65,15,18,1584.38,1584.59,39.0,1486.44,1482.63,0,0,1
1178,42,7,38,1488.72,1483.73,23.0,1561.01,1562.24,0,0,0
1179,9,74,29,1509.61,1514.86,61.0,1393.00,1393.47,0,0,0
1180,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,1


## Retraining the model with the new predicted data

In [194]:
# Preparing Training and Testing Data:
X = Team_id_updated.iloc[:, 0:10].values
y = Team_id_updated.iloc[:, -1].values

X = np.array(X, dtype = 'f')
y = np.array(y, dtype = 'f')

from sklearn.preprocessing import LabelEncoder
LabelEncoder_res=LabelEncoder()
y=LabelEncoder_res.fit_transform(y)

X, y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

##  Rescaling the model with the trained data

In [195]:
# Scalling of Data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Rebuilding an ANN

In [196]:
# Building ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

## Re Seeding

In [197]:
#seed
np.random.seed(47)

## Re-Random Forest

In [199]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rc = RandomForestClassifier(**RFC_tuned_params)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.7372881355932204
0.6056338028169014
              precision    recall  f1-score   support

           0       0.60      0.65      0.62       148
           1       0.60      0.69      0.64       152
           2       0.74      0.25      0.38        55

    accuracy                           0.61       355
   macro avg       0.64      0.53      0.55       355
weighted avg       0.62      0.61      0.59       355

[[ 96  50   2]
 [ 44 105   3]
 [ 21  20  14]]


## Re XGBOOST

In [200]:
# XGBoost

XGB = xgb.XGBClassifier(**tuned_xgb_params)                        
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[09:53:50] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.6997578692493946
0.5943661971830986
              precision    recall  f1-score   support

           0       0.63      0.55      0.59       148
           1       0.58      0.77      0.66       152
           2       0.52      0.24      0.33        55

    accuracy                           0.59       355
   macro avg       0.58      0.52      0.52       355
weighted avg       0.59      0.59      0.58       355

[[ 81  60   7]
 [ 30 117   5]
 [ 17  25  13]]
[CV] END bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=200; total time=   1.1s
[CV] END bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=200; total time=   1.0s
[CV] END bootstrap=True, m

## Repredict

In [201]:
probs, text, ReWrite_pred_df = prediction('USA', 'Brazil',ReWrite_pred_df)
print(text)
ReWrite_pred_df.head()

Chance for USA to win against Brazil is 32.76822865009308
Chance for Brazil to win against USA is 33.25685262680054
Chance for USA and Brazil draw is 33.97492170333862


/tmp/ipykernel_30404/3334352355.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
/tmp/ipykernel_30404/3334352355.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
0,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,1
1,2,5,16,1627.48,1635.01,1,1841.30,1837.56,0,5,0


In [202]:
# K-Fold Cross Vaidation

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True, random_state=47)
kf.get_n_splits(X)
print(kf)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

KFold(n_splits=5, random_state=47, shuffle=True)
TRAIN: [   0    2    3    4    5    6    8    9   11   12   13   14   15   16
   17   18   19   21   22   23   24   26   28   29   30   31   32   35
   36   37   38   39   40   41   42   44   45   46   47   48   49   50
   51   52   53   54   55   56   57   58   59   60   61   62   63   64
   65   66   69   71   72   73   74   76   77   79   81   82   83   84
   85   86   87   88   89   90   92   94   95   96   97   98  101  102
  103  104  105  106  108  109  111  112  113  114  116  117  118  119
  120  121  122  124  125  126  127  128  129  130  132  133  134  135
  136  137  138  140  141  143  144  145  146  147  148  150  151  152
  153  154  155  156  157  159  160  161  162  164  165  167  168  169
  171  172  173  175  176  177  178  179  180  181  182  183  184  185
  188  189  190  191  192  193  194  195  198  199  200  201  202  203
  204  205  207  208  209  210  211  212  213  214  215  216  218  219
  222  223  224  225 

In [203]:
#PCA
from sklearn.decomposition import PCA
pca=PCA(n_components=None)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)